# Mini projects
### DAG's + Airflow 

#### Tasks:

##### 1. Прочитайте csv (ads_data_121288 - ads_data_121288.csv) файл, который находится в этой папке, при помощи библиотеки pandas

##### 2. В данных вы найдете информацию о событиях, которые произошли с объявлением 121288 за два дня. 
##### Рассчитайте следующие метрики в разрезе каждого дня: 
        количество показов,
        количество кликов,
        CTR,
        сумма потраченных денег.
То есть для каждой метрики у вас должно получиться две цифры, за 2019-04-01 и 2019-04-02.
###### Рассчитать сумму потраченных денег можно по следующей формуле:
###### Значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.

##### 3. Теперь найдите процентную разницу между этими метриками. То есть найдите, насколько процентов каждая метрика увеличилась/уменьшилась 2 апреля по сравнению с 1 апреля.



##### 4. Создайте текстовый файл, в котором будет собрана информация о том, какие метрики наблюдаются 2 апреля (представьте, что 2 апреля это сегодня, условно, вы отправляете отчет за сегодня, сравнивая данные со вчера), а также, на сколько процентов они уменьшились по сравнению со вчера. То есть в текстовом файле должен быть список метрик, их значения 2 апреля и процентное отличие от 1 апреля. 

Вариант текстового отчета:
    
    Отчет по объявлению 121288 за 2 апреля
    Траты: Х рублей (+Y%)
    Показы: X (+Y%)
    Клики: X (-Y%)
    CTR: X (-Y%)


##### 5. Отправьте получившийся текст к себе в личные сообщения во ВКонтакте в виде сообщения со сводкой метрик.


##### 6. Все предыдущие шаги оформите в виде исполняемого скрипта (скриптов) и скрипта для DAG-а, в котором при помощи BashOperator-а (или других операторов на ваше усмотрение) будет вызываться исполняемый скрипт (скрипты).


##### 7. В расписании для крона укажите каждый понедельник в 12 утра. Так, чтобы ваш скрипт с рассчетом дневных метрик отправлялся вам в личку каждый понедельник в 12 утра из Airflow

In [2]:
import pandas as pd
import numpy as np


##### 1. Прочитайте csv (ads_data_121288 - ads_data_121288.csv) файл, который находится в этой папке, при помощи библиотеки pandas

In [3]:
path = 'C:/Users/user/Karpov_courses/Аналитик_данных/AirFlow/Mini_pj_DAG/ads_data_121288 - ads_data_121288.csv'
ads  = pd.read_csv(path) 

In [4]:
ads.head()

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
3,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4


In [30]:
duplic_count    = ads.duplicated().sum()
perscent_duplic = round(ads.duplicated().sum() * 100 / ads.shape[0],2)

print(f'''
Количество строк-дубликатов: 
{duplic_count} строк ({perscent_duplic}%)''')


Количество строк-дубликатов: 
12 строк (1.98%)


In [31]:
ads.drop_duplicates(inplace = True)

##### 2. В данных вы найдете информацию о событиях, которые произошли с объявлением 121288 за два дня. 
##### Рассчитайте следующие метрики в разрезе каждого дня: 
        количество показов,
        количество кликов,
        CTR,
        сумма потраченных денег.
То есть для каждой метрики у вас должно получиться две цифры, за 2019-04-01 и 2019-04-02.
###### Рассчитать сумму потраченных денег можно по следующей формуле:
###### Значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.

In [15]:
ads.head()

,date,time,event,platform,ad_id,ad_cost_type,ad_cost
0,2019-04-01,2019-04-01 0:00:48,view,web,121288,CPM,187.4
1,2019-04-01,2019-04-01 0:04:41,view,ios,121288,CPM,187.4
2,2019-04-01,2019-04-01 0:07:50,view,android,121288,CPM,187.4
4,2019-04-01,2019-04-01 0:08:46,view,ios,121288,CPM,187.4
5,2019-04-01,2019-04-01 0:09:36,view,android,121288,CPM,187.4


In [81]:
# количество показов

count_views = ads\
                [ads.event == 'view']\
                .groupby(['date','ad_id'], as_index= False)\
                .agg({'platform':'count'})\
                .rename(columns = {'platform':'count_view'})
count_views

,date,ad_id,count_view
0,2019-04-01,121288,480
1,2019-04-02,121288,92


In [82]:
# количество кликов

count_clicks = ads\
                [ads.event == 'click']\
                .groupby(['date','ad_id'], as_index= False)\
                .agg({'platform':'count'})\
                .rename(columns = {'platform':'count_click'})
count_clicks

,date,ad_id,count_click
0,2019-04-01,121288,17
1,2019-04-02,121288,6


In [83]:
# CTR (click-through rate - коэффициент кликабельности) - отношение числа кликов к числу показов, в процентах

ads_df        = count_clicks.merge(count_views)
ads_df['ctr'] = ads_df.count_click.div(ads_df.count_view).mul(100).round(2) 
ads_df

,date,ad_id,count_click,count_view,ctr
0,2019-04-01,121288,17,480,3.54
1,2019-04-02,121288,6,92,6.52


In [84]:
# Cумма потраченных денег
# формула CRM: значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.

ads_df['ad_cost']   = ads.ad_cost.unique()[0]
ads_df['value_crm'] = ads_df.ad_cost.div(1000).mul(ads_df.count_view).round(2)
ads_df

,date,ad_id,count_click,count_view,ctr,ad_cost,value_crm
0,2019-04-01,121288,17,480,3.54,187.4,89.95
1,2019-04-02,121288,6,92,6.52,187.4,17.24


##### 3. Теперь найдите процентную разницу между этими метриками. То есть найдите, насколько процентов каждая метрика увеличилась/уменьшилась 2 апреля по сравнению с 1 апреля.



In [142]:
# Создание Df с динамикой - мониторинг

ads_monitoring         = ads_df[['date', 'ad_id']]
ads_monitoring['date'] = pd.to_datetime(ads_monitoring.date)

C:\Users\user\AppData\Local\Temp\ipykernel_19628\1919045354.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads_monitoring['date'] = pd.to_datetime(ads_monitoring.date)


In [168]:
 # Метрики за текущий интересующий день

today = '2019-04-02'

value_crm_0204 = float(ads_df[ads_df.date == today].value_crm)
views_0204     = int(ads_df[ads_df.date == today].count_view)
clicks_0204    = int(ads_df[ads_df.date == today].count_click)
ctr_0204       = float(ads_df[ads_df.date == today].ctr)

ads_monitoring = ads_monitoring[ads_monitoring.date == today]

C:\Users\user\AppData\Local\Temp\ipykernel_19628\3004964649.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  value_crm_0204 = float(ads_df[ads_df.date == today].value_crm)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\3004964649.py:3: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  views_0204     = int(ads_df[ads_df.date == today].count_view)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\3004964649.py:4: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  clicks_0204    = int(ads_df[ads_df.date == today].count_click)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\3004964649.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float

In [181]:
# Метрики предыдущего дня

click_yesterday     = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_click)
view_yesterday      = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_view)
ctr_yesterday       = float(ads_df[ads_df.date == ads_df.date.values[-2]].ctr)
value_crm_yesterday = float(ads_df[ads_df.date == ads_df.date.values[-2]].value_crm)

C:\Users\user\AppData\Local\Temp\ipykernel_19628\4066032684.py:1: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  click_yesterday     = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_click)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\4066032684.py:2: FutureWarning: Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead
  view_yesterday      = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_view)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\4066032684.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  ctr_yesterday       = float(ads_df[ads_df.date == ads_df.date.values[-2]].ctr)
C:\Users\user\AppData\Local\Temp\ipykernel_19628\4066032684.py:4: FutureWarning: Calling float on a single element Series is de

In [183]:
# Добавление в Df-мониторинг данных о динамике текущего дня с предыдущим

ads_monitoring['click_dynamic']     = clicks_0204 - click_yesterday
ads_monitoring['view_dynamic']      = views_0204 - view_yesterday
ads_monitoring['ctr_dynamic']       = ctr_0204 - ctr_yesterday
ads_monitoring['value_crm_dynamic'] = value_crm_0204 - value_crm_yesterday

C:\Users\user\AppData\Local\Temp\ipykernel_19628\265970859.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads_monitoring['click_dynamic']     = clicks_0204 - click_yesterday
C:\Users\user\AppData\Local\Temp\ipykernel_19628\265970859.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ads_monitoring['view_dynamic']      = views_0204 - view_yesterday
C:\Users\user\AppData\Local\Temp\ipykernel_19628\265970859.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.

In [226]:
# Пересчет метрик в проценты 

value_crm_dyn_perscent = round((ads_monitoring.value_crm_dynamic.values[0].round(2)/ value_crm_yesterday)*100,2)
views_dyn_perscent     = round((ads_monitoring.view_dynamic.values[0].round(2)/ view_yesterday)*100,2)
clicks_dyn_perscent    = round((ads_monitoring.click_dynamic.values[0].round(2)/ click_yesterday)*100,2)
ctr_dyn_perscent       = round((ads_monitoring.ctr_dynamic.values[0].round(2)/ ctr_yesterday)*100,2)

##### 4. Создайте текстовый файл, в котором будет собрана информация о том, какие метрики наблюдаются 2 апреля (представьте, что 2 апреля это сегодня, условно, вы отправляете отчет за сегодня, сравнивая данные со вчера), а также, на сколько процентов они уменьшились по сравнению со вчера. То есть в текстовом файле должен быть список метрик, их значения 2 апреля и процентное отличие от 1 апреля. 

Вариант текстового отчета:
    
    Отчет по объявлению 121288 за 2 апреля
    Траты: Х рублей (+Y%)
    Показы: X (+Y%)
    Клики: X (-Y%)
    CTR: X (-Y%)

In [213]:
ads_monitoring

,date,ad_id,click_dynamic,view_dynamic,ctr_dynamic,value_crm_dynamic
1,2019-04-02,121288,-11,-388,2.98,-72.71


In [254]:
# Формирование итогового человекочитаемого-отчета

message = (f'''
Отчет по объявлению {ads_monitoring.ad_id.values[0]} за { str(ads_monitoring.date.dt.day[1]) + ' ' + ads_monitoring.date.dt.month_name().values[0]}
Траты:  {value_crm_0204} рублей ({ '+' if value_crm_dyn_perscent > 0 else ''}{value_crm_dyn_perscent} %)
Показы: {views_0204} ({ '+' if views_dyn_perscent > 0 else ''}{views_dyn_perscent} %)
Клики:  {clicks_0204} ({ '+' if clicks_dyn_perscent > 0 else ''}{clicks_dyn_perscent} %)
CTR:    {ctr_0204} ({ '+' if ctr_dyn_perscent > 0 else ''}{ctr_dyn_perscent} %)'''
)

print(message)


Отчет по объявлению 121288 за 2 April
Траты:  17.24 рублей (-80.83 %)
Показы: 92 (-80.83 %)
Клики:  6 (-64.71 %)
CTR:    6.52 (+84.18 %)


##### 5. Отправьте получившийся текст к себе в личные сообщения во ВКонтакте в виде сообщения со сводкой метрик.


In [255]:
# Создание файла report_2019-04-02.txt в директории где найходится текущий файл.ipynb и запись в него сообщения

with open(f'report_{today}.txt', 'w') as file:
    file.write(message)

In [ ]:
# Отправка отчета в ВК

app_token  = '...'
chat_id    = 1
my_id      = 123456789
vk_session = vk_api.VkApi(token = app_token)
vk         = vk_session.get_api()

vk.messagessend(
    chat_id = chat_id,
    random_id = random.randint(1,2,**31),
    message = message
)

##### 6. Все предыдущие шаги оформите в виде исполняемого скрипта (скриптов) и скрипта для DAG-а, в котором при помощи BashOperator-а (или других операторов на ваше усмотрение) будет вызываться исполняемый скрипт (скрипты).


In [ ]:
from datetime import datetime
from airflow import DAG
from airflow.operators.python import PythonOperator

default_args = {
    'owner'            : 'mr_vozhyk',
    'retries'          : 0,
    'start_date'       : datetime(2024, 9, 1),
    'schedule_interval': '00 12 * * 1'
}

dag = DAG('dag_mr_vozhyk_mini_pj',
          default_args = default_args,
          catchup = False)

def send_report_to_vk():
    import pandas as pd
    import numpy as np
    import vk_api
    import random

    today = '2019-04-02'
    
    path  = 'C:/Users/user/Karpov_courses/Аналитик_данных/AirFlow/ads_data_121288 - ads_data_121288.csv'
    ads   = pd.read_csv(path, parse_dates = [0])
    ads.drop_duplicates(inplace = True)
    print('Данные для отчета подгружены')
    
    # количество показов
    count_views = ads\
                    [ads.event == 'view']\
                    .groupby(['date','ad_id'], as_index= False)\
                    .agg({'platform':'count'})\
                    .rename(columns = {'platform':'count_view'})
    # количество кликов
    count_clicks = ads\
                    [ads.event == 'click']\
                    .groupby(['date','ad_id'], as_index= False)\
                    .agg({'platform':'count'})\
                    .rename(columns = {'platform':'count_click'})
    # CTR (click-through rate - коэффициент кликабельности) - отношение числа кликов к числу показов, в процентах
    ads_df        = count_clicks.merge(count_views)
    ads_df['ctr'] = ads_df.count_click.div(ads_df.count_view).mul(100).round(2) 

    # Cумма потраченных денег
    # формула CRM: значение из колонки ad_cost разделить на 1000 и умножить на количество показов объявления.
    ads_df['ad_cost']   = ads.ad_cost.unique()[0]
    ads_df['value_crm'] = ads_df.ad_cost.div(1000).mul(ads_df.count_view).round(2)

    print('Метрики по дням подсчитаны')
    
    # Создание Df с динамикой - мониторинг
    ads_monitoring         = ads_df[['date', 'ad_id']]
    ads_monitoring['date'] = pd.to_datetime(ads_monitoring.date)
    
    # Метрики за текущий интересующий день
    value_crm_0204 = float(ads_df[ads_df.date == today].value_crm)
    views_0204     = int(ads_df[ads_df.date == today].count_view)
    clicks_0204    = int(ads_df[ads_df.date == today].count_click)
    ctr_0204       = float(ads_df[ads_df.date == today].ctr)
    
    # Оставление в Df-мониторинге только текущего дня
    ads_monitoring = ads_monitoring[ads_monitoring.date == today]
    
    # Метрики предыдущего дня
    click_yesterday     = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_click)
    view_yesterday      = int(ads_df[ads_df.date == ads_df.date.values[-2]].count_view)
    ctr_yesterday       = float(ads_df[ads_df.date == ads_df.date.values[-2]].ctr)
    value_crm_yesterday = float(ads_df[ads_df.date == ads_df.date.values[-2]].value_crm)
    
    # Добавление в Df-мониторинг данных о динамике текущего дня с предыдущим
    ads_monitoring['click_dynamic']     = clicks_0204 - click_yesterday
    ads_monitoring['view_dynamic']      = views_0204 - view_yesterday
    ads_monitoring['ctr_dynamic']       = ctr_0204 - ctr_yesterday
    ads_monitoring['value_crm_dynamic'] = value_crm_0204 - value_crm_yesterday

    # Пересчет метрик в проценты 
    value_crm_dyn_perscent = round((ads_monitoring.value_crm_dynamic.values[0].round(2)/ value_crm_yesterday)*100,2)
    views_dyn_perscent     = round((ads_monitoring.view_dynamic.values[0].round(2)/ view_yesterday)*100,2)
    clicks_dyn_perscent    = round((ads_monitoring.click_dynamic.values[0].round(2)/ click_yesterday)*100,2)
    ctr_dyn_perscent       = round((ads_monitoring.ctr_dynamic.values[0].round(2)/ ctr_yesterday)*100,2)

    # Формирование итогового человекочитаемого-отчета
    message = (f'''
        Отчет по объявлению {ads_monitoring.ad_id.values[0]} за { str(ads_monitoring.date.dt.day[1]) + ' ' + ads_monitoring.date.dt.month_name().values[0]}
        Траты:  {value_crm_0204} рублей ({ '+' if value_crm_dyn_perscent > 0 else ''}{value_crm_dyn_perscent} %)
        Показы: {views_0204} ({ '+' if views_dyn_perscent > 0 else ''}{views_dyn_perscent} %)
        Клики:  {clicks_0204} ({ '+' if clicks_dyn_perscent > 0 else ''}{clicks_dyn_perscent} %)
        CTR:    {ctr_0204} ({ '+' if ctr_dyn_perscent > 0 else ''}{ctr_dyn_perscent} %)'''
        )
    print('Отчет сформирован')
    
    # Создание файла report_{today}.txt и запись в него сообщения
    with open(f'report_{today}.txt', 'w') as file:
        file.write(message)
    print(f'Отчет сохранен под именем report_{today}.txt')

    # Отправка отчета в ВК
    app_token  = '...'
    chat_id    = 1
    my_id      = 123456789
    vk_session = vk_api.VkApi(token = app_token)
    vk         = vk_session.get_api()
    
    vk.messagessend(
        chat_id   = chat_id,
        random_id = random.randint(1,2,**31),
        message   = message
    )
    print('Отчет отправлен')

##### 7. В расписании для крона укажите каждый понедельник в 12 утра. Так, чтобы ваш скрипт с рассчетом дневных метрик отправлялся вам в личку каждый понедельник в 12 утра из Airflow

In [ ]:
'schedule_interval': '00 12 * * 1'